## 创建Tensor

"tensor"这个单词一般可译作“张量”，张量可以看作是一个多维数组。标量可以看作是0维张量，向量可以看作1维张量，矩阵可以看作是二维张量。

In [2]:
import torch

初始化一个张量

In [12]:
#创建一个5x3的未初始化的Tensor
x = torch.empty(5, 3)
print(x)
#创建一个5x3的随机初始化的Tensor
x = torch.rand(5, 3)
print(x)
#创建一个5x3的long型全0的Tensor
x = torch.zeros(5, 3, dtype=torch.long)
print(x)
#直接根据数据创建
x = torch.tensor([5.5, 3])
print(x)
print(x.dtype) #默认是float32
#还可以通过现有的Tensor来创建，此方法会默认重用输入Tensor的一些属性，例如数据类型，除非自定义数据类型。
x = x.new_ones(5, 3, dtype=torch.double)
print(x)

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
tensor([[0.0516, 0.9107, 0.1519],
        [0.2697, 0.0395, 0.9326],
        [0.0055, 0.9031, 0.2970],
        [0.4191, 0.0706, 0.0036],
        [0.2803, 0.5047, 0.1042]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([5.5000, 3.0000])
torch.float32
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)


Tensor运算，要注意如下几点：
- y = x + y，y的对象地址会被改变，也就是说，Tensor运算后，会生成一个新的Tensor对象，而不是在原有的Tensor对象上进行运算。
- y += x，y的对象地址不会改变，而是在原有的Tensor对象上进行运算。
- 可以用函数实现对应基本运算，注意PyTorch操作inplace版本都有后缀_, 例如x.copy_(y), x.t_(), y.add_(x)。

In [19]:
x = torch.tensor(range(1,16), dtype=torch.int64).reshape(5,3)
y = torch.rand(5, 3)
print(x+y)
print(torch.add(x, y))
#加法：方式2
result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)
#加法：原地
y.add_(x)
print(y)

tensor([[ 1.3820,  2.5705,  3.3092],
        [ 4.7309,  5.0341,  6.6097],
        [ 7.6664,  8.5918,  9.5258],
        [10.3557, 11.6479, 12.8001],
        [13.1058, 14.5395, 15.5730]])
tensor([[ 1.3820,  2.5705,  3.3092],
        [ 4.7309,  5.0341,  6.6097],
        [ 7.6664,  8.5918,  9.5258],
        [10.3557, 11.6479, 12.8001],
        [13.1058, 14.5395, 15.5730]])
tensor([[ 1.3820,  2.5705,  3.3092],
        [ 4.7309,  5.0341,  6.6097],
        [ 7.6664,  8.5918,  9.5258],
        [10.3557, 11.6479, 12.8001],
        [13.1058, 14.5395, 15.5730]])
tensor([[ 1.3820,  2.5705,  3.3092],
        [ 4.7309,  5.0341,  6.6097],
        [ 7.6664,  8.5918,  9.5258],
        [10.3557, 11.6479, 12.8001],
        [13.1058, 14.5395, 15.5730]])


索引

- tensor直接变量赋值（=）是引用。
- 用[]索引出来的结果与原数据共享内存。
- 深拷贝：clone()，不共享内存。

In [23]:
m1=torch.tensor([[1,2],[3,4]])
m2=m1
m3=m2[0,:]
m3+=1
print(m1)

tensor([[2, 3],
        [3, 4]])


改变形状

- view()，改变形状，共享内存。
- reshape()，改变形状，不共享内存（此函数并不能保证返回的是其拷贝，所以不推荐使用）。
- item()，将一个标量Tensor转换成一个Python number。

In [24]:
x_cp = x.clone().view(15)
x -= 1
print(x)
print(x_cp)

tensor([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11],
        [12, 13, 14]])
tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])


广播（复制）机制

两个张量的形状不同时，PyTorch会自动触发广播机制：
- 先适当复制元素使这两个张量形状相同。然后按元素运算。
- 两个张量形状在某个维度上不匹配，但是如果有一个张量这个维度的长度为1，则可以利用广播机制进行计算。否则报错。

In [43]:
x = torch.tensor([1, 2, 3,4]).view(1, 2, 2)
y = torch.tensor([1,1,1,1]).view(2, 2, 1)
print(x+y) #2x2x2，x的第一个维度复制了一次，y的第三个维度复制了一次

tensor([[[2, 3],
         [4, 5]],

        [[2, 3],
         [4, 5]]])


In [44]:
x = torch.tensor([1, 2, 3,4]).view(1, 4, 1)
y = torch.tensor([1,1,1,1]).view(2, 2, 1)
print(x+y) #报错，第二个维度不一样且都不为1
torch.norm(x, 2, dim=1)

RuntimeError: The size of tensor a (4) must match the size of tensor b (2) at non-singleton dimension 1

Tensor转NumPy
- 使用numpy()

NumPy转Tensor
- 使用torch.from_numpy()，注意：该函数产生的的Tensor与NumPy数组共享内存，当修改其中一个时，另一个也会被修改。
- 使用torch.tensor()，注意：该函数总是会进行数据拷贝，返回的Tensor和原来的数据不再共享内存。

Tensor on GPU
- 使用.to(device)将Tensor移动到GPU上。

In [ ]:
# 以下代码只有在PyTorch GPU版本上才会执行
if torch.cuda.is_available():
    device = torch.device("cuda")          # GPU
    y = torch.ones_like(x, device=device)  # 直接创建一个在GPU上的Tensor
    x = x.to(device)                       # 等价于 .to("cuda")
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # to()还可以同时更改数据类型